# Лекция 4: Продвинутые методы Pandas

**Курс:** Введение в машинное обучение

---

### Цели лекции

После освоения базовых операций мы переходим к более сложным и мощным инструментам, которые необходимы для подготовки реальных данных к анализу и построению моделей.

1.  **Научиться работать с отсутствующими данными (missing data):** освоить методы их обнаружения (`.isnull()`), удаления (`.dropna()`) и заполнения (`.fillna()`).
2.  **Освоить агрегацию данных с помощью `GROUP BY`:** понять, как группировать данные по категориям и считать для них различные статистики (среднее, сумму, количество).
3.  **Изучить метод `.apply()`:** научиться применять собственные функции к столбцам для сложных преобразований данных.
4.  **Научиться объединять таблицы:** рассмотреть два основных способа объединения `DataFrames` — `pd.concat()` для простого "склеивания" и `pd.merge()` для "умного" соединения по ключам, как в SQL.

Эти навыки составляют основу работы Data Scientist'а и являются критически важными для решения подавляющего большинства задач по обработке данных.

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

## 1. Отсутствующие данные (Missing Data)

В реальных наборах данных практически всегда встречаются пропуски. Они могут возникать из-за ошибок сбора данных, неполных записей или просто потому, что какая-то информация была недоступна. Большинство алгоритмов машинного обучения не умеют работать с пропусками, поэтому их необходимо обработать.

В Pandas пропущенные значения обычно представляются как `NaN` (Not a Number).

**Основные стратегии работы с пропусками:**
1.  **Оставить как есть:** Подходит, если ваш следующий инструмент (например, некоторые продвинутые модели) умеет работать с пропусками. (Плюс: не вносим искажений. Минус: мало где применимо).
2.  **Удалить:** Можно удалить либо строки, либо столбцы с пропусками. (Плюс: просто. Минус: теряем данные, что может быть критично).
3.  **Заполнить:** Заменить пропуски некоторым значением (например, нулем, средним, медианой или модой). (Плюс: сохраняем данные. Минус: вносим предположения, которые могут исказить результаты).

Создадим небольшой DataFrame для демонстрации.

In [2]:
data = {'A': [1, 2, np.nan, 4],
        'B': [5, np.nan, np.nan, 8],
        'C': [9, 10, 11, 12]}
df = pd.DataFrame(data)

In [5]:
df.describe()

,A,B,C
count,3.000000,2.00000,4.000000
mean,2.333333,6.50000,10.500000
std,1.527525,2.12132,1.290994
min,1.000000,5.00000,9.000000
25%,1.500000,5.75000,9.750000
50%,2.000000,6.50000,10.500000
75%,3.000000,7.25000,11.250000
max,4.000000,8.00000,12.000000


### 1.1. Обнаружение пропусков: `.isnull()`
Метод `.isnull()` возвращает DataFrame такой же размерности с булевыми значениями, где `True` означает пропуск.

In [6]:
df.isnull()

,A,B,C
0,False,False,False
1,False,True,False
2,True,True,False
3,False,False,False


Чтобы быстро посчитать количество пропусков в каждом столбце, можно применить метод `.sum()` к результату `.isnull()`.

In [8]:
df.isnull().any()

A     True
B     True
C    False
dtype: bool

### 1.2. Удаление пропусков: `.dropna()`
Самый простой способ — удалить строки или столбцы с пропусками.

In [9]:
# Удаление любой строки, содержащей хотя бы один пропуск (NaN)
df.dropna()

,A,B,C
0,1.0,5.0,9
3,4.0,8.0,12


In [10]:
df

,A,B,C
0,1.0,5.0,9
1,2.0,NaN,10
2,NaN,NaN,11
3,4.0,8.0,12


In [11]:
# Удаление любого столбца, содержащего хотя бы один пропуск
df.dropna(axis=1)

,C
0,9
1,10
2,11
3,12


In [12]:
# Использование параметра thresh (порог)
# Оставить только те строки, в которых есть как минимум 2 не-пропущенных значения
df.dropna(thresh=2)

,A,B,C
0,1.0,5.0,9
1,2.0,NaN,10
3,4.0,8.0,12


### 1.3. Заполнение пропусков: `.fillna()`
Более гибкий подход — заполнение пропусков.

In [13]:
# Заполнение всех пропусков одним значением, например, нулем
df.fillna(0)

,A,B,C
0,1.0,5.0,9
1,2.0,0.0,10
2,0.0,0.0,11
3,4.0,8.0,12


Часто пропуски заполняют средним значением по столбцу. Это позволяет сохранить общую статистику данных.

In [18]:
# Заполнение пропусков в столбце 'A' средним значением этого столбца
mean_A = df['A'].mean()
print(mean_A)
df['A'].fillna(mean_A)

2.3333333333333335


0    1.000000
1    2.000000
2    2.333333
3    4.000000
Name: A, dtype: float64

In [17]:
df


,A,B,C
0,1.0,5.0,9
1,2.0,NaN,10
2,NaN,NaN,11
3,4.0,8.0,12


## 2. Агрегация данных: Group By

Операция `Group By` (группировка) — одна из самых мощных в Pandas. Она позволяет реализовать парадигму **Split-Apply-Combine** (Разделить-Применить-Объединить):

1.  **Split:** Данные разделяются на группы на основе некоторого категориального признака.
2.  **Apply:** К каждой группе независимо применяется некоторая функция (например, `sum`, `mean`, `count`).
3.  **Combine:** Результаты применения функции объединяются в новый DataFrame.

Это позволяет отвечать на вопросы вроде: "Какова средняя цена товара в каждой категории?" или "Какова суммарная выручка по каждому городу?"

Для примеров будем использовать датасет `mpg.csv`.

In [20]:
df_cars = pd.read_csv('mpg.csv')
df_cars.head()
df_cars

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


### Группировка и применение агрегирующей функции

Давайте найдем средние значения характеристик автомобилей для каждого года выпуска (`model_year`).

In [21]:
# Шаг 1 и 2: Группируем по 'model_year' и считаем среднее для каждой группы
# Нечисловые столбцы (как 'name') автоматически игнорируются
avg_by_year = df_cars.groupby('model_year').mean(numeric_only=True)
avg_by_year.head()

,mpg,cylinders,displacement,weight,acceleration,origin
model_year,,,,,,
70,17.689655,6.758621,281.413793,3372.793103,12.948276,1.310345
71,21.250000,5.571429,209.750000,2995.428571,15.142857,1.428571
72,18.714286,5.821429,218.375000,3237.714286,15.125000,1.535714
73,17.100000,6.375000,256.875000,3419.025000,14.312500,1.375000
74,22.703704,5.259259,171.740741,2877.925926,16.203704,1.666667


Можно сгруппировать данные и затем выбрать один столбец для агрегации.

In [22]:
# Среднее значение 'mpg' для каждого года
avg_mpg_by_year = df_cars.groupby('model_year')['mpg'].mean()
avg_mpg_by_year.head()

model_year
70    17.689655
71    21.250000
72    18.714286
73    17.100000
74    22.703704
Name: mpg, dtype: float64

Можно использовать и другие функции, например, `.describe()` для получения полной статистики по группам.

In [23]:
df_cars.groupby('cylinders').describe()['mpg']

,count,mean,std,min,25%,50%,75%,max
cylinders,,,,,,,,
3,4.0,20.550000,2.564501,18.0,18.75,20.25,22.05,23.7
4,204.0,29.286765,5.710156,18.0,25.00,28.25,33.00,46.6
5,3.0,27.366667,8.228204,20.3,22.85,25.40,30.90,36.4
6,84.0,19.985714,3.807322,15.0,18.00,19.00,21.00,38.0
8,103.0,14.963107,2.836284,9.0,13.00,14.00,16.00,26.6


## 3. Метод `.apply()`

Метод `.apply()` — это ваш "швейцарский нож" для тех случаев, когда встроенных функций Pandas не хватает. Он позволяет применить любую вашу функцию к каждому элементу столбца (Series).

Предположим, мы хотим классифицировать автомобили по расходу топлива.

In [25]:
# Шаг 1: Создаем нашу функцию
def classify_mpg(mpg):
    if mpg < 15:
        return 'Очень низкий'
    elif 15 <= mpg < 25:
        return 'Средний'
    else:
        return 'Высокий'

In [26]:
# Шаг 2: Применяем функцию к столбцу 'mpg' и создаем новый столбец
df_cars['mpg_class'] = df_cars['mpg'].apply(classify_mpg)
df_cars[['name', 'mpg', 'mpg_class']].head()

,name,mpg,mpg_class
0,chevrolet chevelle malibu,18.0,Средний
1,buick skylark 320,15.0,Средний
2,plymouth satellite,18.0,Средний
3,amc rebel sst,16.0,Средний
4,ford torino,17.0,Средний


То же самое можно сделать короче с помощью **lambda-функции**:

In [28]:
# Пример с lambda-функцией. Создадим столбец, указывающий, тяжелый ли автомобиль
df_cars['is_heavy'] = df_cars['weight'].apply(lambda w: 'Да' if w > 3000 else 'Нет')
df_cars[['name', 'weight', 'is_heavy']].tail()

,name,weight,is_heavy
393,ford mustang gl,2790,Нет
394,vw pickup,2130,Нет
395,dodge rampage,2295,Нет
396,ford ranger,2625,Нет
397,chevy s-10,2720,Нет


In [30]:
data1 = {'Good': ['water', 'cake', 'shoes', 'bag', 'tax','bus','gum','magnum'],
        'Price': [250, 600, 40000, 43000, 4000, 4200, 350, 20000],
        'Gaterogy': ['Food','Food', 'Clothes', 'Accessory', 'Transport', 'Transport', 'Food','Food'],
       }
df1 = pd.DataFrame(data1)

In [33]:
df1.head()

,Good,Price,Gaterogy
0,water,250,Food
1,cake,600,Food
2,shoes,40000,Clothes
3,bag,43000,Accessory
4,tax,4000,Transport


In [36]:
df_by_category = df1.groupby('Gaterogy').sum(numeric_only = True)

In [37]:
df_by_category

,Price
Gaterogy,
Accessory,43000
Clothes,40000
Food,21200
Transport,8200


In [38]:
def class_for_price(x):
    if x>20000:
        return 'Expensive'
    elif x>10000:
        return 'Medium'
    else:
        return 'Cheap'

In [44]:
df1['Price_class_bylambda'] = df1['Price'].apply(lambda x: 'Expensive' if x>20000 else 'Cheap')
df1['Price_class_bydef'] = df1['Price'].apply(class_for_price)

In [48]:
df1 = df1.drop("Price_class", axis=1)

In [ ]:
df1.drop("Price_class", axis=1, inplace=Tru

In [49]:
df1

,Good,Price,Gaterogy,Price_class_bylambda,Price_class_bydef
0,water,250,Food,Cheap,Cheap
1,cake,600,Food,Cheap,Cheap
2,shoes,40000,Clothes,Expensive,Expensive
3,bag,43000,Accessory,Expensive,Expensive
4,tax,4000,Transport,Cheap,Cheap
5,bus,4200,Transport,Cheap,Cheap
6,gum,350,Food,Cheap,Cheap
7,magnum,20000,Food,Cheap,Medium


## 4. Объединение DataFrame

Часто данные, которые вам нужны, хранятся в нескольких разных файлах или таблицах. Pandas предоставляет мощные инструменты для их объединения.

### 4.1. `pd.concat()`

`concat` (конкатенация) — это простое "склеивание" таблиц по вертикали или по горизонтали. Главное условие — таблицы должны иметь совместимую структуру.

Представим, что у нас есть данные за два разных года в двух файлах.

In [50]:
df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']},
                     index=[0, 1])

df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']},
                     index=[2, 3])
print(df1)
print(df2)

    A   B
0  A0  B0
1  A1  B1
    A   B
2  A2  B2
3  A3  B3


In [51]:
# Вертикальное объединение (по умолчанию axis=0)
pd.concat([df1, df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [52]:
# Горизонтальное объединение
pd.concat([df1, df2], axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


### 4.2. `pd.merge()`

`merge` — это более интеллектуальное объединение, аналогичное `JOIN` в SQL. Оно соединяет таблицы на основе значений в одном или нескольких общих столбцах (ключах).

Создадим два DataFrame: один с данными о регистрации, другой — о входе в систему.

In [53]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [54]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [55]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


#### Inner Merge (внутреннее объединение)
Это объединение по умолчанию. В результат попадают только те строки, для которых ключи (`name`) есть в **обеих** таблицах.

In [56]:
# Pandas автоматически находит общий столбец 'name'
pd.merge(registrations, logins, how='inner', on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


#### Left Merge (левое объединение)
В результат попадают **все** строки из левой таблицы (`registrations`) и только совпадающие строки из правой. Если совпадения нет, значения для столбцов из правой таблицы заполняются `NaN`.

In [59]:
pd.merge(registrations, logins, how='left', on='name')
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

## Резюме и следующие шаги

*   **Что мы узнали сегодня:**
    *   Три стратегии работы с пропусками: оставить, удалить (`.dropna()`) или заполнить (`.fillna()`).
    *   Как работает механизм `Split-Apply-Combine` в `groupby` для агрегации данных.
    *   Как применять свои собственные функции к данным с помощью `.apply()`.
    *   Как "склеивать" (`.concat()`) и "соединять" (`.merge()`) таблицы.

*   **Что дальше?**
    *   На **семинаре** мы отработаем каждую из этих техник на практических примерах.
    *   В **лабораторной работе** вам предстоит применить эти методы для более глубокого анализа набора данных.